In [ ]:
# 必要モジュールのインポート
import os
from dotenv import load_dotenv

# .envファイルの内容を読み込見込む
load_dotenv()

# スター数上位100のjavascriptリポジトリのURLを取得

In [1]:
import requests
import json

def fetch_top_github_repos():
    url = "https://api.github.com/search/repositories"
    headers = {
        "Accept": "application/vnd.github+json",
        "User-Agent": "Top-Repo-Fetcher"
    }
    params = {
        "q": "stars:>1 language:JavaScript",  # JavaScriptでスター数が1以上のリポジトリを検索
        "sort": "stars",   # スター数でソート
        "order": "desc",  # 降順
        "per_page": 100     # 1ページの最大件数
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        # リポジトリのURLを抽出
        top_repos = [repo["html_url"] for repo in data.get("items", [])]
        
        # JSON形式で保存
        with open("top100_javascript_repositories.json", "w", encoding="utf-8") as f:
            json.dump(top_repos, f, ensure_ascii=False, indent=4)

        print("トップ100のJavaScriptリポジトリのURLが 'top100_javascript_repositories.json' に保存されました。")

    except requests.exceptions.RequestException as e:
        print(f"APIリクエストに失敗しました: {e}")

if __name__ == "__main__":
    fetch_top_github_repos()


トップ100のJavaScriptリポジトリのURLが 'top100_javascript_repositories.json' に保存されました。


# libraries.ioからキーワードを取得したい

In [1]:
import json
import requests
import time

# APIトークンを設定（ここに取得したトークンを記入）
API_TOKEN = os.environ["LIBRALIESIO_API_KEY"]

# 入力ファイルと出力ファイルのパス
INPUT_FILE = "top100_javascript_repositories.json"
OUTPUT_FILE = "keywords.json"

# ベースURL
BASE_URL = "https://libraries.io/api"

def get_library_keywords(github_url):
    """
    指定されたGitHubリポジトリURLに一致するライブラリのキーワードを取得します。
    """
    try:
        # APIリクエストURLを作成
        url = f"{BASE_URL}/search"
        params = {
            "q": github_url,
            "api_key": API_TOKEN
        }
        
        # APIにリクエストを送信
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # APIのレスポンスを解析
        results = response.json()
        if results:
            # 最初の結果のキーワードを返す
            return results[0].get("keywords", [])
        else:
            return []
    except Exception as e:
        print(f"Error fetching keywords for {github_url}: {e}")
        return []

def main():
    # 入力ファイルを読み込み
    with open(INPUT_FILE, "r") as infile:
        github_urls = json.load(infile)
    
    # 結果を格納する辞書
    keywords_dict = {}
    
    for github_url in github_urls:
        print(f"Fetching keywords for {github_url}...")
        keywords = get_library_keywords(github_url)
        keywords_dict[github_url] = keywords
        time.sleep(1)
    
    # 結果を出力ファイルに保存
    with open(OUTPUT_FILE, "w") as outfile:
        json.dump(keywords_dict, outfile, indent=4)
    
    print(f"Keywords saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


Fetching keywords for https://github.com/facebook/react...
Fetching keywords for https://github.com/trekhleb/javascript-algorithms...
Fetching keywords for https://github.com/twbs/bootstrap...
Fetching keywords for https://github.com/airbnb/javascript...
Fetching keywords for https://github.com/vercel/next.js...
Fetching keywords for https://github.com/Chalarangelo/30-seconds-of-code...
Fetching keywords for https://github.com/nodejs/node...
Fetching keywords for https://github.com/axios/axios...
Fetching keywords for https://github.com/mrdoob/three.js...
Fetching keywords for https://github.com/facebook/create-react-app...
Fetching keywords for https://github.com/ryanmcdermott/clean-code-javascript...
Fetching keywords for https://github.com/iptv-org/iptv...
Fetching keywords for https://github.com/microsoft/Web-Dev-For-Beginners...
Fetching keywords for https://github.com/sveltejs/svelte...
Fetching keywords for https://github.com/jaywcjlove/awesome-mac...
Fetching keywords for https

# npmのキーワード（keywords=serverで検索したときの）を取りたい

In [2]:
import requests
from bs4 import BeautifulSoup
import time

def get_list_items(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        ul_elements = soup.find_all('ul', class_='cf33f2b9 w-90 list mh0 mv2 pa0 truncate')

        all_items = []
        for ul in ul_elements[:10]:  # 最初の10個の<ul>に対して処理
            li_elements = ul.find_all('li')  # <li>要素をすべて取得
            item_texts = [li.get_text(strip=True) for li in li_elements]  # 文字を取得
            all_items.append(item_texts)
            time.sleep(1)
        
        return all_items
    else:
        print("ページの取得に失敗しました。")
        return []

# 使用例
url = 'https://www.npmjs.com/search?page=0&q=keywords%3Aserver&sortBy=dependent_count
'
list_items = get_list_items(url)

# 結果の表示
for index, items in enumerate(list_items):
    print(f"リスト {index + 1}:")
    for item in items:
        print(f" - {item}")


リスト 1:
 - react
 - framework
 - nextjs
 - web
 - server
 - node
 - front-end
 - backend
 - cli
 - vercel
リスト 2:
 - util
 - functional
 - server
 - client
 - browser
リスト 3:
 - mysql
 - client
 - server
リスト 4:
 - task
 - async
 - cli
 - minify
 - uglify
 - build
 - lodash
 - unit
 - test
 - qunit
 - nodeunit
 - server
 - init
 - scaffold
 - View more
リスト 5:
 - static
 - file
 - server
リスト 6:
 - websocket
 - websockets
 - socket
 - networking
 - comet
 - push
 - RFC-6455
 - realtime
 - server
 - client
リスト 7:
 - database
 - mssql
 - sql
 - server
 - msnodesql
 - sqlserver
 - tds
 - node-tds
 - tedious
 - node-sqlserver
 - msnodesqlv8
 - azure
 - node-mssql
リスト 8:
 - model
 - view
 - controller
 - router
 - server
 - client
 - browser
リスト 9:
 - cli
 - command
 - static
 - http
 - https
 - http-server
 - https-server
 - server
リスト 10:
 - validation
 - validate
 - server
 - client


In [ ]:
API_TOKEN = os.environ["LIBRALIESIO_API_KEY"]

In [2]:
import requests
import json
import time

# ベースURLの設定
BASE_URL = 'https://libraries.io/api/platforms/npm/projects'

# 出力ファイル名の設定
OUTPUT_FILE = 'top_js_rep_key.json'

# 1回のリクエストで取得するプロジェクト数
PER_PAGE = 50

# APIリクエスト間の待機時間（秒）
REQUEST_DELAY = 1

def fetch_projects(page):
    """指定したページのプロジェクト一覧を取得する"""
    params = {
        'api_key': API_KEY,
        'sort': 'stars',
        'per_page': PER_PAGE,
        'page': page
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'APIリクエスト失敗: {response.status_code}, {response.text}')

def main():
    all_projects = []
    page = 1

    while len(all_projects) < 100:
        print(f'ページ {page} のデータを取得中...')
        projects = fetch_projects(page)
        if not projects:
            break
        all_projects.extend(projects)
        page += 1
        time.sleep(REQUEST_DELAY)

    # 上位100件に絞る
    top_projects = all_projects[:100]

    # 必要な情報を抽出
    result = []
    for project in top_projects:
        result.append({
            'name': project['name'],
            'repository_url': project['repository_url'],
            'keywords': project.get('keywords', [])
        })

    # JSONファイルに保存
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

    print(f'データを {OUTPUT_FILE} に保存しました。')

if __name__ == '__main__':
    main()



ページ 1 のデータを取得中...


Exception: APIリクエスト失敗: 404, <!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="msvalidate.01" content="3667D2FEB238C1BF174F3CB1AE0D3C16" />
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="/assets/application-54c51e436e998966c20a8e883979835b548519abf02da82fdbe6c604f2205a16.css" media="all" data-turbolinks-track="true" />
  <title>Libraries.io - security & maintenance data for open source software</title>
  <meta name="description" content="Discover open source packages, modules and frameworks you can use in your code.">
  <meta name="csrf-param" content="authenticity_token" />
<meta name="csrf-token" content="PSvue1H7o7yF3es_n5rH7I6sXcUdKsJE966nIgAdBH1WUgv_VCk0wF_9-uhzGil-dqUw1zpqFgdOfn0GeCTnVg" />
  
  <link rel="apple-touch-icon-precomposed" sizes="57x57" href="https://libraries.io/assets/apple-touch-icon-57x57-d2ce1ae09b96b2dcd4548cdab571b2f0f04567fd52bb42acdc158f3e499fb9f

# リンク整形

In [7]:
import json
import os

def git_url(json_file):
    url_list = []
    # JSONファイルを読み込む
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for item in data:
        # 各行からURLを取得
        url_list.append(item)

    return url_list

def create_npm_url(json_file):
    url_list = []
    # JSONファイルを読み込む
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for item in data:
        if(item != ''):
            # 各行からnpmのURLを作成
            url = "https://www.npmjs.com/package/" + item
            url_list.append(url)
        else:
            url_list.append(item)

    return url_list

# ファイルパスの設定
file_path = 'jsProject_path.json'

# 結果を格納するリスト
result = []

git_list = []
npm_list = []

 # JSONファイルのパスを指定
github_file = './top100_javascript_repositories.json' 
npm_file = './jsTopRep_npmURL.json'

git_list = git_url(github_file)
npm_list = create_npm_url(npm_file)


for (g, n) in zip(git_list, npm_list): 
    # JSONデータを作成
    result.append({
        "github": g,
        "npm": n,
        "libraries.io": ""
    })

# JSONファイルに書き込む
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)
    
print("finish!")

finish!


# キーワード取得（github npm libraries.io）

In [6]:
import requests
import time
from bs4 import BeautifulSoup
import json
import os

# githubのトピックを取得する
def get_repository_topics(repository, token):
    rep_split = repository.split('/')
    owner = rep_split[-2]
    repo = rep_split[-1]
    
    url = f"https://api.github.com/repos/{owner}/{repo}/topics"
    # ヘッダーを設定
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.mercy-preview+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        topics = response.json().get("names", [])
    elif response.status_code == 404:
        print("リポジトリが見つかりません。")
    else:
        print(f"エラーが発生しました: {response.status_code} - {response.text}")
    
    return(topics)


# npmのキーワードを取得する
def npm_keywords(url):
    # キーワードを格納するリスト
    keywords = []
    
    if(url != ''):
        # HTMLコンテンツを取得
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # <li class="dib mr2">を検索
        for li in soup.select('li.dib.mr2'):
            # <a>タグのテキストを取得
            a_tag = li.find('a')
            if a_tag and a_tag.text:
                keywords.append(a_tag.text.strip())
    else:
        pass
    
    return keywords


# libraries.ioのキーワードを取得する
def libraries_keywords(url):
    # キーワードを格納するリスト
    keywords = []
    
    if(url != ''):
        # HTMLコンテンツを取得
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # <dt>Keywords</dt>の次の<dd>を取得
        dt_keywords = soup.find('dt', string=lambda text: text and 'Keywords' in text.strip())
        if dt_keywords:
            dd_keywords = dt_keywords.find_next_sibling('dd')
            if dd_keywords:
                # <a>タグのテキストを取得
                for a_tag in dd_keywords.find_all('a'):
                    keywords.append(a_tag.text.strip())
    else:
        pass

    return keywords
    
i=0

# ファイルパスの設定
file_path = 'keywords_V2.json'
# トークン
token = os.environ['GITHUB_API_KEY']

# 結果を格納するリスト
result = []

url_list = []
json_file = 'jsProject_path.json'  # JSONファイルのパスを指定

# JSONファイルを読み込む
with open(json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

for item in data:
    # プロジェクト名作成
    rep_split = item['github'].split('/')
    clone_dir = rep_split[-2] + "_" + rep_split[-1]
    
    github = []
    npm = []
    libraries = []
    
    github = get_repository_topics(item['github'], token)
    npm = npm_keywords(item['npm'])
    libraries_keywords(item['libraries.io'])
    
    # 要素を重複させずにリストを結合する
    keyword_list = list(set(github + npm + libraries))
    keyword_list.sort()
    
    # JSONデータを作成
    result.append({
        "Name": clone_dir,
        "keywords": keyword_list
    })
    
    i += 1
    print(str(i) + " " + clone_dir)
    time.sleep(3)


# JSONファイルに書き込む
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

1 facebook_react
2 trekhleb_javascript-algorithms
3 twbs_bootstrap
4 airbnb_javascript
5 vercel_next.js
6 Chalarangelo_30-seconds-of-code
7 nodejs_node
8 axios_axios
9 mrdoob_three.js
10 facebook_create-react-app
11 ryanmcdermott_clean-code-javascript
12 iptv-org_iptv
13 microsoft_Web-Dev-For-Beginners
14 sveltejs_svelte
15 jaywcjlove_awesome-mac
16 FortAwesome_Font-Awesome
17 typicode_json-server
18 anuraghazra_github-readme-stats
19 hakimel_reveal.js
20 expressjs_express
21 chartjs_Chart.js
22 webpack_webpack
23 leonardomso_33-js-concepts
24 resume_resume.github.com
25 louislam_uptime-kuma
26 atom_atom
27 lodash_lodash
28 adam-p_markdown-here
29 jquery_jquery
30 angular_angular.js
31 h5bp_html5-boilerplate
32 gatsbyjs_gatsby
33 scutan90_DeepLearning-500-questions
34 azl397985856_leetcode
35 jgraph_drawio-desktop
36 Semantic-Org_Semantic-UI
37 juliangarnier_anime
38 prettier_prettier
39 mozilla_pdf.js
40 chinese-poetry_chinese-poetry
41 moment_moment
42 gorhill_uBlock
43 marktext_mark

# 各リポジトリの説明を取得

In [3]:
import requests
import time
from bs4 import BeautifulSoup
import json
import os

# GitHubのAbout情報を取得する関数
def github_abouts(url):
    if url:
        try:
            # HTMLコンテンツを取得
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # ステータスコードがエラーの場合、例外を発生させる
            soup = BeautifulSoup(response.text, 'html.parser')

            # <div class="hide-sm hide-md">を検索
            for div in soup.select('div.hide-sm.hide-md'):
                # <p>タグのテキストを取得
                p_tag = div.find('p')
                if p_tag and p_tag.text:
                    description = p_tag.text.strip()
        except requests.RequestException as e:
            print(f"Error fetching URL {url}: {e}")
    return description

# ファイルパスの設定
file_path = 'repository_V4.json'
json_file = 'jsProject_path.json'  # プロジェクトリストのJSONファイル


# 結果を格納するリスト
result = []
i=0

# JSONファイルを読み込む
with open(json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

# keywords_V2.jsonの読み込み
with open('keywords_V2.json', 'r', encoding='utf-8') as file:
    keywords_data = json.load(file)

    
for item in data:
    # プロジェクト名作成
    rep_split = item['github'].split('/')
    clone_dir = rep_split[-2] + "_" + rep_split[-1]
    
    # About情報を取得
    about = github_abouts(item['github'])
    
    for key in keywords_data:
        if key['Name'] == clone_dir:
                result.append({
                    'Name':clone_dir,
                    'keywords':key['keywords'],
                    'About':about,
                    'efficiency':0
                })
    
                i += 1
                print(str(i) + " " + clone_dir)
                print(key['keywords'])
                print(about)
                print()
    
    # APIリクエスト制限を避けるために待機
    time.sleep(2)

# JSONファイルに書き込む
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

1 facebook_react
['declarative', 'frontend', 'javascript', 'library', 'react', 'ui']
The library for web and native user interfaces.

2 trekhleb_javascript-algorithms
['algorithm', 'algorithms', 'computer-science', 'cs', 'data-structures', 'graph', 'interview', 'interview-preparation', 'javascript', 'javascript-algorithms', 'sorting-algorithms', 'tree']
📝 Algorithms and data structures implemented in JavaScript with explanations and links to further readings

3 twbs_bootstrap
['bootstrap', 'css', 'css-framework', 'framework', 'front-end', 'html', 'javascript', 'mobile-first', 'responsive', 'sass', 'scss', 'web']
The most popular HTML, CSS, and JavaScript framework for developing responsive, mobile first projects on the web.

4 airbnb_javascript
['airbnb', 'arrow-functions', 'es2015', 'es2016', 'es2017', 'es2018', 'es6', 'eslint', 'javascript', 'jsx', 'lint', 'linting', 'naming-conventions', 'react', 'style guide', 'style-guide', 'style-linter', 'styleguide', 'tc39']
JavaScript Style Gu

42 gorhill_uBlock
['adblock', 'blocker', 'browser-extension', 'chromium', 'firefox', 'javascript', 'trie', 'uBO', 'uBlock', 'ublock', 'ublock-origin']
uBlock Origin - An efficient blocker for Chromium and Firefox. Fast and lean.

43 marktext_marktext
['dark-mode', 'editor', 'electron', 'element-ui', 'emoji', 'focus-mode', 'latex', 'linux', 'mac', 'macos', 'markdown', 'marktext', 'next-generation', 'source-code', 'typewriter-mode', 'vue', 'windows']
📝A simple and elegant markdown editor, available for Linux, macOS and Windows.

44 TryGhost_Ghost
['blogging', 'cms', 'creator-economy', 'ghost', 'hacktoberfest', 'headless-cms', 'jamstack', 'javascript', 'journalism', 'nodejs', 'publishing', 'web-application']
Independent technology for modern publishing, memberships, subscriptions and newsletters.

45 NARKOZ_hacker-scripts
[]
Based on a true story

46 cypress-io_cypress
['angular-testing-library', 'automation', 'browser', 'component', 'component-testing', 'cypress', 'cypress-cloud', 'cypre

73 NaiboWang_EasySpider
['batch-processing', 'batch-script', 'code-free', 'crawler', 'data-collection', 'frontend', 'gui', 'html', 'input-parameters', 'layman', 'parameters', 'robotics', 'rpa', 'scraper', 'spider', 'visual', 'visualization', 'visualprogramming', 'web', 'www']
A visual no-code/code-free web crawler/spider易采集：一个可视化浏览器自动化测试/数据采集/爬虫软件，可以无代码图形化的设计和执行爬虫任务。别名：ServiceWrapper面向Web应用的智能化服务封装系统。

74 naptha_tesseract.js
['deep-learning', 'javascript', 'ocr', 'tesseract', 'webassembly']
Pure Javascript OCR for more than 100 Languages 📖🎉🖥

75 denysdovhan_wtfjs
['book', 'guide', 'handbook', 'javascript', 'js', 'learning', 'notes', 'specification', 'wtf']
🤪 A list of funny and tricky JavaScript examples

76 alvarotrigo_fullPage.js
['fullpage', 'fullpagejs', 'fullscreen', 'javacript', 'javascript', 'jquery', 'mouse', 'mousewheel', 'one_page', 'onepage', 'scroll', 'scrolling', 'sections', 'single_page', 'slide', 'slides', 'slideshow', 'sliding', 'snap', 'swipe', 'wheel']
fullPage plugin

# 重複するキーワードだけ集めてみる

In [15]:
import json
from collections import Counter

# JSONファイルの読み込み
with open('keywords.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 全てのkeywordsを収集
all_keywords = []
for item in data:
    all_keywords.extend(item.get('keywords', []))

# 重複するkeywordsをカウント
keyword_counts = Counter(all_keywords)

# 2つ以上重複するkeywordsをフィルタリング
duplicate_keywords = {keyword for keyword, count in keyword_counts.items() if count > 1}

# 重複するkeywordsを持つ元のデータをフィルタリング（repositoryも含む）
filtered_data = []
for item in data:
    repository = item.get('Name')  # repositoryの取得
    filtered_keywords = [keyword for keyword in item.get('keywords', []) if keyword in duplicate_keywords]

    # keywordsが重複していない場合や空値の場合
    if not filtered_keywords:
        filtered_data.append({
            'repository': repository,
            'keywords': []  # 空値を保存
        })
    else:
        filtered_data.append({
            'repository': repository,
            'keywords': filtered_keywords
        })

# 結果を新しいJSONファイルに書き込む
with open('keywords_dup.json', 'w', encoding='utf-8') as file:
    json.dump(filtered_data, file, ensure_ascii=False, indent=4)


In [17]:
import json

json_file = 'keywords_dup.json'

# JSONファイルを読み込む
with open(json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

i = 1
for item in data:
    print(i)
    print(item)
    print()
    i += 1

1
{'repository': 'facebook_react', 'keywords': ['javascript', 'library', 'react', 'ui']}

2
{'repository': 'trekhleb_javascript-algorithms', 'keywords': ['algorithm', 'algorithms', 'computer-science', 'data-structures', 'graph', 'interview', 'javascript', 'tree']}

3
{'repository': 'twbs_bootstrap', 'keywords': ['css', 'css-framework', 'framework', 'front-end', 'html', 'javascript', 'responsive', 'scss', 'web']}

4
{'repository': 'airbnb_javascript', 'keywords': ['es2015', 'es6', 'javascript', 'jsx', 'react']}

5
{'repository': 'vercel_next.js', 'keywords': ['blog', 'browser', 'cli', 'compiler', 'components', 'framework', 'front-end', 'node', 'react', 'server', 'ssg', 'static-site-generator', 'web']}

6
{'repository': 'Chalarangelo_30-seconds-of-code', 'keywords': ['awesome-list', 'css', 'education', 'es6-javascript', 'git', 'html', 'javascript', 'nodejs', 'programming']}

7
{'repository': 'nodejs_node', 'keywords': ['javascript', 'js', 'linux', 'macos', 'node', 'nodejs', 'runtime', 'w

In [19]:
len(duplicate_keywords)

91

# jsonからxlsxへ

In [3]:
import json
import os
import pandas as pd


# プロジェクトディレクトリの一覧
RESULT_DIR = "./zenkoku_result"
result = os.listdir(RESULT_DIR)

# 移動
os.chdir('zenkoku_result')

for dir in result:
    # JSONファイルの読み込み
    with open(dir, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # データフレームの作成
    df = pd.DataFrame(data)

    # 必要な列を指定して並べ替え
    df = df[['repository', 'fast', 'slow']]

    # Excelファイルの作成
    fname = dir.split('.')[0]
    df.to_excel(f'{fname}.xlsx', index=False, engine='openpyxl')
    print(f'create {fname}.xlsx')

# 移動
os.chdir('..')


create for_forEach_TOP100.xlsx
